In [2]:
# turnquest_advanced_v3.py
import pygame, sys, random
from dataclasses import dataclass, field
from typing import List, Optional, Callable, Dict

# -------------------- Config --------------------
WIDTH, HEIGHT = 1400, 800
FPS = 60
FONT_NAME = None
BG_COLOR = (18, 20, 28)
PANEL = (28, 34, 45)
ACCENT = (99, 155, 255)
TEXT = (230, 230, 240)
HP_GREEN = (94, 221, 121)
HP_RED = (235, 92, 92)
LOG_BG = (14, 16, 20)
BUTTON = (48, 58, 75)
BUTTON_HOVER = (70, 95, 140)
DISABLED = (70, 70, 80)

pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("TurnQuest Advanced v3")
clock = pygame.time.Clock()
font = pygame.font.Font(FONT_NAME, 18)
bigfont = pygame.font.Font(FONT_NAME, 28)
title_font = pygame.font.Font(FONT_NAME, 46)

# -------------------- Data Models --------------------
@dataclass
class Status:
    name: str
    duration: int
    potency: int = 0

@dataclass
class Move:
    name: str
    desc: str
    power: int = 0
    accuracy: float = 1.0
    cooldown: int = 0
    current_cd: int = 0
    target_self: bool = False
    apply_status: Optional[Callable[['Character','Character'], Optional[Status]]] = None

    def ready(self):
        return self.current_cd <= 0

    def trigger_cd(self):
        if self.cooldown > 0:
            self.current_cd = self.cooldown

    def tick_cd(self):
        if self.current_cd > 0:
            self.current_cd -= 1

@dataclass
class Item:
    name: str
    desc: str
    use: Callable[['Character'], str]
    qty: int = 1

@dataclass
class Character:
    name: str
    level: int
    max_hp: int
    hp: int
    atk: int
    defense: int
    speed: int
    moves: List[Move]
    statuses: List[Status] = field(default_factory=list)
    buffs: Dict[str, Status] = field(default_factory=dict)
    xp: int = 0

    def is_alive(self):
        return self.hp > 0

    def take_damage(self, amt: int):
        amt = max(0, amt)
        self.hp = max(0, min(self.max_hp, self.hp - amt))
        return amt

    def heal(self, amt: int):
        if amt <= 0:
            return 0
        healed = max(0, min(self.max_hp - self.hp, amt))
        self.hp += healed
        return healed

    def add_status(self, s: Status):
        for ex in self.statuses:
            if ex.name == s.name:
                ex.duration = s.duration
                ex.potency = s.potency
                return
        self.statuses.append(s)

    def has_status(self, name: str):
        return any(s.name == name for s in self.statuses)

    def tick_statuses(self):
        msgs = []
        for s in self.statuses[:]:
            if s.name in ['Burn','Poison']:
                dmg = s.potency
                self.take_damage(dmg)
                msgs.append(f"{self.name} takes {dmg} {s.name.lower()} damage.")
            elif s.name == 'Stun':
                msgs.append(f"{self.name} is stunned and skips turn!")
            elif s.name == 'Defense Down':
                msgs.append(f"{self.name}'s defense is lowered!")
            s.duration -= 1
            if s.duration <= 0:
                self.statuses.remove(s)
                msgs.append(f"{self.name} is no longer {s.name.lower()}.")
        for k,b in list(self.buffs.items()):
            b.duration -= 1
            if b.duration <= 0:
                del self.buffs[k]
                msgs.append(f"{self.name}'s {k} buff wore off.")
        return msgs

    def get_stat(self, stat: str):
        base = getattr(self, stat)
        buff = self.buffs.get(stat)
        if buff:
            return base + buff.potency
        return base

    def add_buff(self, stat: str, amount: int, duration: int):
        self.buffs[stat] = Status(name=f"buff:{stat}", duration=duration, potency=amount)

    def tick_move_cds(self):
        for m in self.moves:
            m.tick_cd()

# -------------------- Game Data --------------------
def make_player_moves():
    return [
        Move("Slash","A solid physical strike.",18,0.95,0),
        Move("Power Strike","Heavy hit, lower accuracy.",30,0.75,2),
        Move("First Aid","Heal yourself for moderate HP.",-25,1.0,3,True),
        Move("Flame Shot","Deal fire damage and may burn.",15,0.9,3,
             apply_status=lambda u,t: Status("Burn",3,5))
    ]

def make_enemy_moves(level):
    return [
        Move("Bite","Enemy bite.",12+level*2,0.95),
        Move("Rage","Powerful blow.",20+level*3,0.8,2),
        Move("Spit","May burn target.",8+level,0.85,
             apply_status=lambda u,t: Status("Burn",3,4)),
        Move("Poison Fang","May poison.",6+level,0.9,
             apply_status=lambda u,t: Status("Poison",3,3))
    ]

def make_items():
    return {
        'potion': Item("Potion","Heals 40 HP.",qty=3,use=lambda c:f"{c.name} healed {c.heal(40)} HP."),
        'antidote': Item("Antidote","Removes Burn/Poison.",qty=1,use=lambda c:_use_antidote(c))
    }

def _use_antidote(c):
    removed=False
    for s in c.statuses[:]:
        if s.name in ['Burn','Poison']:
            c.statuses.remove(s)
            removed=True
    return f"{c.name} had their status removed." if removed else f"{c.name} had no status to remove."

def calculate_damage(attacker,defender,base_power):
    atk = attacker.get_stat('atk')
    defense = max(1,defender.get_stat('defense'))
    raw=(atk*base_power)/defense
    variation=random.uniform(0.85,1.15)
    return int(max(1,raw*variation))

def attempt_hit(move):
    return random.random() <= move.accuracy

def enemy_choose_move(enemy,player):
    usable=[m for m in enemy.moves if m.ready()]
    if not usable: return None
    if enemy.hp < enemy.max_hp*0.35:
        return sorted(usable,key=lambda mm: mm.power)[0]
    candidates=sorted(usable,key=lambda mm: mm.power,reverse=True)
    if random.random()<0.2:
        for m in usable:
            if m.apply_status: return m
    return candidates[0]

def generate_enemy(player_level):
    lvl=max(1,player_level+random.choice([-1,0,1]))
    base_hp=60+lvl*10
    return Character(
        name=random.choice(['Goblin','Wolf','Bandit','Slime','Rogue']),
        level=lvl,
        max_hp=base_hp,
        hp=base_hp,
        atk=8+lvl*2,
        defense=4+lvl,
        speed=5+lvl,
        moves=make_enemy_moves(lvl)
    )

# -------------------- UI Widget --------------------
class Button:
    def __init__(self,rect,text,action=None,font=font,enabled=True):
        self.rect=pygame.Rect(rect)
        self.text=text
        self.action=action
        self.font=font
        self.enabled=enabled
        self.hover=False
    def draw(self,surf):
        col=BUTTON
        if not self.enabled: col=DISABLED
        elif self.hover: col=BUTTON_HOVER
        pygame.draw.rect(surf,col,self.rect,border_radius=8)
        txt=self.font.render(self.text,True,TEXT)
        surf.blit(txt,txt.get_rect(center=self.rect.center))
    def handle_event(self,ev):
        if ev.type==pygame.MOUSEMOTION:
            self.hover=self.rect.collidepoint(ev.pos)
        elif ev.type==pygame.MOUSEBUTTONDOWN and ev.button==1:
            if self.enabled and self.rect.collidepoint(ev.pos):
                if self.action: self.action()

# -------------------- Game Class --------------------
class TurnQuestGame:
    def __init__(self):
        self.state='start'
        self.player_name="Hero"
        self.player=None
        self.enemy=None
        self.inventory=None
        self.log=[]
        self.encounters=0
        self.selected_move=None
        self.message_timer=0.0
        self.anim_t=0.0
        self.death_count=0
        self.ui_buttons=[]
        self.move_buttons=[]
        self.item_buttons=[]
        self.quit_button=None
        self.continue_button=None
        self.rest_button=None
        self.setup_start_screen()

    # Start screen
    def setup_start_screen(self):
        self.input_active=True
        self.input_text=""
        self.name_box=pygame.Rect(WIDTH//2-250,HEIGHT//2-30,500,48)
        self.start_btn=Button((WIDTH//2-120,HEIGHT//2+40,120,48),"Start",action=self.start_game)
        self.exit_btn=Button((WIDTH//2+10,HEIGHT//2+100,120,48),"Quit",action=self.quit_game)

    def start_game(self):
        name=self.input_text.strip() or "Hero"
        self.player_name=name
        self.player=Character(name=self.player_name,level=1,max_hp=120,hp=120,
                              atk=14,defense=9,speed=10,moves=make_player_moves())
        self.inventory=make_items()
        self.encounters=0
        self.log=[]
        self.next_encounter()
        self.state='battle'

    def quit_game(self):
        pygame.quit()
        sys.exit(0)

    # Next encounter and move buttons in columns of 4
    def next_encounter(self):
        self.enemy=generate_enemy(self.player.level)
        self.log.append(f"A wild {self.enemy.name} appears!")
        cols=4
        btn_w,btn_h=200,48
        start_x,start_y=50,500
        gap_x,gap_y=20,10
        self.ui_buttons=[]
        self.move_buttons=[]
        for idx,m in enumerate(self.player.moves):
            col=idx%cols
            row=idx//cols
            x=start_x+col*(btn_w+gap_x)
            y=start_y+row*(btn_h+gap_y)
            btn=Button((x,y,btn_w,btn_h),m.name,action=lambda mv=m:self.player_attack(mv))
            self.move_buttons.append(btn)
            self.ui_buttons.append(btn)
        self.quit_button=Button((WIDTH-140,60,100,40),"Quit",action=self.quit_game)
        self.ui_buttons.append(self.quit_button)

    # Player attack
    def player_attack(self,move):
        if not move.ready():
            self.log.append(f"{move.name} is on cooldown {move.current_cd} turn(s).")
            return
        move.trigger_cd()
        if attempt_hit(move):
            target=self.enemy if not move.target_self else self.player
            dmg=move.power if move.power>=0 else -target.heal(-move.power)
            if move.power>0:
                dmg=calculate_damage(self.player,target,move.power)
                target.take_damage(dmg)
            self.log.append(f"{self.player.name} used {move.name} for {dmg} damage!")
            if move.apply_status:
                st=move.apply_status(self.player,target)
                if st:
                    target.add_status(st)
                    self.log.append(f"{target.name} is affected by {st.name}!")
        else:
            self.log.append(f"{self.player.name}'s {move.name} missed!")
        self.tick_round_end()

    # Enemy turn with damage boost
    def enemy_turn(self):
        move=enemy_choose_move(self.enemy,self.player)
        if not move:
            self.log.append(f"{self.enemy.name} has no available moves!")
            return
        move.trigger_cd()
        if attempt_hit(move):
            target=self.player if not move.target_self else self.enemy
            dmg=move.power if move.power>=0 else -target.heal(-move.power)
            if move.power>0:
                dmg=int(calculate_damage(self.enemy,target,move.power)*1.3)  # Enemy stronger
                target.take_damage(dmg)
            self.log.append(f"{self.enemy.name} used {move.name} for {dmg} damage!")
            if move.apply_status:
                st=move.apply_status(self.enemy,target)
                if st:
                    target.add_status(st)
                    self.log.append(f"{target.name} is affected by {st.name}!")
        else:
            self.log.append(f"{self.enemy.name}'s {move.name} missed!")
        self.tick_round_end()

    # Tick round end
    def tick_round_end(self):
        for msg in self.player.tick_statuses(): self.log.append(msg)
        for msg in self.enemy.tick_statuses(): self.log.append(msg)
        self.player.tick_move_cds()
        self.enemy.tick_move_cds()
        if not self.enemy.is_alive():
            self.handle_victory()
        elif not self.player.is_alive():
            self.handle_defeat()
        else:
            if random.random()<0.5:
                self.enemy_turn()

    # Victory/defeat and leveling
    def handle_victory(self):
        xp_gain=self.enemy.level*15
        self.player.xp+=xp_gain
        self.log.append(f"{self.player.name} gains {xp_gain} XP!")
        self.check_level_up()
        self.next_encounter()

    def handle_defeat(self):
        self.death_count+=1
        self.player.hp=self.player.max_hp
        self.player.level=max(1,self.player.level-1)
        self.player.moves=make_player_moves()
        self.log.append(f"{self.player.name} was revived at full HP. Deaths: {self.death_count}")
        self.next_encounter()

    def check_level_up(self):
        leveled=False
        xp_needed=50*self.player.level
        while self.player.xp>=xp_needed:
            self.player.xp-=xp_needed
            self.player.level+=1
            self.player.max_hp+=12
            self.player.hp=self.player.max_hp
            self.player.atk+=3
            self.player.defense+=2
            self.player.speed+=1
            leveled=True
            self.log.append(f"{self.player.name} leveled up to {self.player.level}!")
            self.try_upgrade_move()
            xp_needed=50*self.player.level
        return leveled

    # Move upgrades with extra moves
    def try_upgrade_move(self):
        extra_moves=[
            Move("Ice Spike","May freeze the enemy.",12,0.9,0,
                 apply_status=lambda u,t: Status("Stun",1)),
            Move("Poison Dart","May poison the enemy.",10,0.95,0,
                 apply_status=lambda u,t: Status("Poison",3,3)),
            Move("Shield Up","Increase own defense.",0,1,0,True,
                 apply_status=lambda u,t: u.add_buff("defense",5,3) or None),
            Move("Fireball","Burns the enemy.",15,0.9,0,
                 apply_status=lambda u,t: Status("Burn",3,5)),
            Move("Thunder Strike","Chance to stun.",18,0.85,0,
                 apply_status=lambda u,t: Status("Stun",1)),
            Move("Vampiric Bite","Heals user for part of damage.",12,0.9,0),
            Move("Poison Cloud","Poisons the enemy over time.",10,0.9,0,
                 apply_status=lambda u,t: Status("Poison",3,4))
        ]
        if self.player.level in [3,5,7,10]:
            if len(self.player.moves)<6 and random.random()<0.5:
                new_mv=random.choice(extra_moves)
                self.player.moves.append(new_mv)
                self.log.append(f"{self.player.name} learned a new move: {new_mv.name}!")
            else:
                mv=random.choice(self.player.moves)
                mv.power+=5
                self.log.append(f"{mv.name} grew stronger! Power is now {mv.power}.")

    # Event handling
    def handle_events(self,ev):
        if self.state=='start':
            if ev.type==pygame.KEYDOWN:
                if self.input_active:
                    if ev.key==pygame.K_RETURN:
                        self.start_game()
                    elif ev.key==pygame.K_BACKSPACE:
                        self.input_text=self.input_text[:-1]
                    else:
                        self.input_text+=ev.unicode
            self.start_btn.handle_event(ev)
            self.exit_btn.handle_event(ev)
        elif self.state=='battle':
            for b in self.ui_buttons:
                b.handle_event(ev)

    def update(self,dt):
        pass

    def draw(self,surf):
        surf.fill(BG_COLOR)
        if self.state=='start':
            txt=title_font.render("TurnQuest Advanced",True,ACCENT)
            surf.blit(txt,txt.get_rect(center=(WIDTH//2,HEIGHT//4)))
            pygame.draw.rect(surf,PANEL,self.name_box,border_radius=8)
            txt2=font.render(self.input_text or "Enter hero name",True,TEXT)
            surf.blit(txt2,(self.name_box.x+8,self.name_box.y+10))
            self.start_btn.draw(surf)
            self.exit_btn.draw(surf)
        elif self.state=='battle':
            # Player info
            ph=self.player.hp/self.player.max_hp
            pygame.draw.rect(surf,HP_RED,(50,120,400,28),border_radius=6)
            pygame.draw.rect(surf,HP_GREEN,(50,120,int(400*ph),28),border_radius=6)
            surf.blit(font.render(f"{self.player.name} Lv {self.player.level} HP:{self.player.hp}/{self.player.max_hp}",True,TEXT),(50,90))
            # Enemy info
            eh=self.enemy.hp/self.enemy.max_hp
            pygame.draw.rect(surf,HP_RED,(WIDTH-500,120,400,28),border_radius=6)
            pygame.draw.rect(surf,HP_GREEN,(WIDTH-500,120,int(400*eh),28),border_radius=6)
            surf.blit(font.render(f"{self.enemy.name} Lv {self.enemy.level} HP:{self.enemy.hp}/{self.enemy.max_hp}",True,TEXT),(WIDTH-500,90))
            # Death counter
            surf.blit(font.render(f"Deaths: {self.death_count}",True,ACCENT),(WIDTH-140,18))
            # Battle log
            pygame.draw.rect(surf,LOG_BG,(28,220,WIDTH-56,250),border_radius=10)
            for i,line in enumerate(self.log[-10:]):
                surf.blit(font.render(line,True,TEXT),(38,420 - i*24))
            # Buttons
            for b in self.ui_buttons:
                b.draw(surf)

# -------------------- Main --------------------
def main():
    game=TurnQuestGame()
    while True:
        dt=clock.tick(FPS)/1000.0
        for ev in pygame.event.get():
            if ev.type==pygame.QUIT:
                pygame.quit()
                sys.exit(0)
            game.handle_events(ev)
        game.update(dt)
        game.draw(screen)
        pygame.display.flip()

if __name__=="__main__":
    main()


SystemExit: 0